In [1]:
import os
import jieba
import jieba.analyse
import pandas as pd
import pyodbc
import configparser
from bs4 import BeautifulSoup
import re
import nltk

In [2]:
config = configparser.ConfigParser()
config.read('config.env')
db_UserName = config.get('DEFAULT', 'DB_USERNAME')
db_Password = config.get('DEFAULT', 'DB_PASSWORD')
db_Name = config.get('DEFAULT', 'DB_NAME')
db_Host = config.get('DEFAULT', 'DB_HOST')

cnxn_str = ("Driver={ODBC Driver 17 for SQL Server};"
            f"Server={db_Host};"
            f"Database={db_Name};"
            f"UID={db_UserName};"
            f"PWD={db_Password};")

cnxn = pyodbc.connect(cnxn_str)
# Create a cursor from the connection
cursor = cnxn.cursor()

In [3]:
# SQL查詢語句
query = ("select source,id,title ,context from ("
         "select a.source,a.id,title,context from pttpost_referendum_2 a "
         " inner join pttpost b on a.source=b.source and a.id=b.Id "
         " where not exists (select * from keyword where source=99 and (b.title like '%'+keyname+'%' or b.context like '%'+keyname+'%')) "
         " union all "
         " select a.source,a.id,title,context from pttpost_referendum_2 a "
         " inner join pttpostgossing b on a.source=b.source and a.id=b.Id "
         " where not exists (select * from keyword where source=99 and (b.title like '%'+keyname+'%' or b.context like '%'+keyname+'%')) "
         " union all "
         " select a.source,convert(varchar,a.id),title,content from dcard.dbo.pttpost_referendum_2 a "
         " inner join dcard.dbo.post b on a.source=b.forum and a.id=b.Id "
         " where not exists (select * from keyword where source=99 and (b.title like '%'+keyname+'%' or b.content like '%'+keyname+'%')) "         
         " ) m "
         "where 1=1")

# 讀取資料表
df = pd.read_sql(query, cnxn)

c:\Users\HuanChen\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [4]:
# 載入知網詞庫
jieba.set_dictionary('C:\project\python\dict.big5.txt')

# 載入自定義詞庫
jieba.load_userdict('C:\project\python\main.txt')

Building prefix dict from C:\project\python\dict.big5.txt ...
Loading model from cache C:\Users\HuanChen\AppData\Local\Temp\jieba.u7bf78fb8a3e5c528afaa2a9a1de33675.cache
Loading model cost 0.792 seconds.
Prefix dict has been built successfully.


In [5]:
def get_score(article_string):
    with open('NTUSD_positive_unicode.txt', encoding='utf-8', mode='r') as f:
        positive_words = []
        for l in f:
            positive_words.append(l.strip())

    with open('NTUSD_negative_unicode.txt', encoding='utf-8', mode='r') as f:
        negative_words = []
        for l in f:
            negative_words.append(l.strip())

    score = 0

    if (article_string.strip() != ""):
        jieba_result = jieba.cut(article_string, cut_all=False, HMM=True)

        for word in jieba_result:
            if word.strip() != "":
                if word in positive_words:
                    score += 1
                elif word in negative_words:
                    score -= 1
                else:
                    pass

    return score

In [6]:
# 迴圈處理
try:
    for index, row in df.iterrows():
        context = row["context"]
        # 進行相關處理
        score = get_score(context)
        print(row["id"]+f'總分:{score}')
        if (row["source"]!="talk"):
            sql = "update pttpost_referendum_2 set emotionpost=? where source=? and id=?"
        else:
            sql = "update dcard.dbo.pttpost_referendum_2 set emotionpost=? where source=? and id=?"    
        
        cursor.execute(sql, score, row["source"], row["id"])
        cnxn.commit()
finally:
    cnxn.commit()
    cnxn.close()

M.1609733351.A.BC4總分:0
M.1609516575.A.7ED總分:0
M.1609595113.A.FAA總分:0
M.1609666034.A.4A1總分:0
M.1609776407.A.088總分:0
M.1610095255.A.9CE總分:0
M.1609978031.A.3D1總分:0
M.1609757399.A.188總分:0
M.1609905715.A.732總分:0
M.1609814601.A.837總分:0
M.1610888330.A.CBF總分:0
M.1610190593.A.A3E總分:0
M.1610220000.A.BE8總分:0
M.1610248992.A.BE4總分:0
M.1610347624.A.FA6總分:0
M.1610118271.A.C6D總分:0
M.1610152714.A.B17總分:0
M.1610342545.A.286總分:0
M.1610508814.A.6A5總分:0
M.1610959744.A.7D0總分:0
M.1610533058.A.7CA總分:0
M.1610581400.A.1E4總分:0
M.1610951166.A.BD0總分:0
M.1610694841.A.BC8總分:0
M.1610498949.A.2D3總分:0
M.1610070191.A.8BE總分:0
M.1610802688.A.597總分:0
M.1610878157.A.E60總分:0
M.1610107284.A.5C7總分:0
M.1610982373.A.051總分:0
M.1611118776.A.259總分:0
M.1610864253.A.D0D總分:0
M.1610934901.A.B18總分:0
M.1611206747.A.BF8總分:0
M.1611341045.A.A13總分:0
M.1610019454.A.E14總分:0
M.1610521445.A.229總分:0
M.1611319117.A.69C總分:0
M.1611487424.A.D7C總分:0
M.1610876085.A.035總分:0
M.1610777837.A.835總分:0
M.1611672968.A.EAF總分:0
M.1611071691.A.2E4總分:0
M.161115828